In [101]:
import polars as pl
import pandas as pd
import numpy as np 
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from tqdm import tqdm
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.preprocessing import LabelEncoder

In [102]:
df = pl.read_parquet(r'C:\Users\310\Desktop\Progects_Py\data\microstructure_price_prediction_data\cross_section\df_cross_section_V0.1_.parquet')

Note that here we are going to use our precomputed features and some raw inputs. We are going to drop "target" (use target_one_step_ahead instead).

In [103]:
df = df.select(pl.exclude("target"))


In [104]:
train_test_ratio = 0.1


df_train = pl.DataFrame()
df_test = pl.DataFrame()

for currency_pair in df["symbol"].unique():

    curr_df = df.filter(pl.col('symbol') == currency_pair)
    split_indx = int(len(curr_df) * train_test_ratio)
    
    train = curr_df[:split_indx]
    test = curr_df[split_indx:]

    print(f'Train len for {currency_pair} is {len(train)}')
    print(f'Test len for {currency_pair} is {len(test)}')

    df_train = df_train.vstack(train) if not df_train.is_empty() else train
    df_test = df_test.vstack(test) if not df_test.is_empty() else test

df_train_pd = df_train.to_pandas()

Train len for AVAX-USDT is 275832
Test len for AVAX-USDT is 2482491
Train len for DOGE-USDT is 731709
Test len for DOGE-USDT is 6585382


In [105]:
print(df_train_pd)

                     trade_time     price  quantity  is_buyer_maker  \
0       2024-06-01 00:00:08.502  36.04000      6.43           False   
1       2024-06-01 00:00:08.511  36.05000      8.40           False   
2       2024-06-01 00:00:08.513  36.05000     24.72           False   
3       2024-06-01 00:00:08.514  36.05000      1.90           False   
4       2024-06-01 00:00:08.515  36.05000     32.55           False   
...                         ...       ...       ...             ...   
1007536 2024-06-08 17:53:42.426   0.14550   1000.00            True   
1007537 2024-06-08 17:53:42.427   0.14550   6382.00            True   
1007538 2024-06-08 17:53:42.898   0.14548   4000.00            True   
1007539 2024-06-08 17:53:42.935   0.14548     53.00           False   
1007540 2024-06-08 17:53:45.698   0.14546   2100.00            True   

              date     symbol       quote  last_ask  last_bid   spread  ...  \
0       2024-06-01  AVAX-USDT   231.73720       NaN  36.04000      N

In [106]:

df_train_pd = df_train_pd.set_index(['symbol', 'trade_time'])

#X = df_train.select(pl.exclude('target_one_step_ahead')).to_pandas()
#y = df_train.select(pl.col("target_one_step_ahead")).to_pandas()


In [117]:
df_train_pd.groupby("trade_time")["target_one_step_ahead"].mean()

trade_time
2024-06-01 00:00:03.927    0.158985
2024-06-01 00:00:06.777    0.158985
2024-06-01 00:00:07.152    0.158985
2024-06-01 00:00:07.156    0.158985
2024-06-01 00:00:07.405    0.158990
                             ...   
2024-06-08 17:53:42.426    0.145500
2024-06-08 17:53:42.427    0.145490
2024-06-08 17:53:42.898    0.145480
2024-06-08 17:53:42.935    0.145470
2024-06-08 17:53:45.698    0.145470
Name: target_one_step_ahead, Length: 1002177, dtype: float64

In [ ]:
#df_train = df_train.with_columns([
#    pl.struct(['symbol', 'trade_time']).alias('multi_index')
#])

#le = LabelEncoder()
#df_train['multi_index'] = le.fit_transform(df_train['multi_index'])

#df_train = df_train.select(pl.exclude("trade_time", "date", "symbol", "is_buyer_maker",))



#X = df_train.select(pl.exclude('target_one_step_ahead')).to_pandas().dropna()
#y = df_train.select(pl.col("target_one_step_ahead")).to_pandas().dropna()
#

In [112]:
X = df_train_pd.drop(columns=["target_one_step_ahead"])
y = df_train_pd["target_one_step_ahead"]


In [113]:
progress = tqdm(total=1, desc="Training Random Forest")

# Initialize Random Forest with hyperparameters
rf = RandomForestRegressor(
    n_estimators=100,  # Default, adjust if needed
    criterion='squared_error',
    max_depth=10,      # Maximum depth of random tree
    min_samples_split=50,  # Minimum number of samples required to split an internal node
    min_samples_leaf=20, #Minimum number of samples required to be at a leaf node
    min_weight_fraction_leaf=0.2, #Minimum weighted fraction of the sum of weights required at a leaf node
    #max_features='auto', #Number of features considered when looking for the best split
    max_leaf_nodes=100, #Maximum number of leaf nodes in a tree
    min_impurity_decrease=0.01,
    random_state=42,
    n_jobs=-1,         # Use all cores for faster training
    verbose=2
)

cv_scores = cross_val_score(rf, X.head(1000), y.head(1000), cv=5, scoring="neg_mean_squared_error")

rf.fit(X.head(1000), y.head(1000))
progress.update(1)
progress.close()

# Results
print(f"Cross-Validation Scores: {cv_scores}")
print(f"Mean CV R²: {np.mean(cv_scores)}")

Training Random Forest:   0%|          | 0/1 [00:00<?, ?it/s]

ValueError: 
All the 5 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\310\AppData\Local\pypoetry\Cache\virtualenvs\microstructure-price-prediction-p_py7spM-py3.12\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\310\AppData\Local\pypoetry\Cache\virtualenvs\microstructure-price-prediction-p_py7spM-py3.12\Lib\site-packages\sklearn\base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\310\AppData\Local\pypoetry\Cache\virtualenvs\microstructure-price-prediction-p_py7spM-py3.12\Lib\site-packages\sklear